# Installing Libraries

In [1]:
#!pip install pycaret
import pandas as pd
from pycaret.regression import *

# Getting Dataset and Model

In [2]:
sih_model = load_model('./Model/sih_motor_xgboost_efficiency_regressor')
pv1 = input("3 Phase Voltage in Volts: ")
pi1 = input("3 Phase Current in Amps: ")
po1 = input("Poles: ")
f1 = input("Frequency in Hz: ")
pf1 = input("Power Factor: ")
t1 = input("Temperature in Celcius: ")
l1 = input("% Load: ")
w1 = input("Angular Velocity in rad/sec: ")
a1 = input("Motor Age in Yrs: ")

if len(pv1)==0:
    pv1 = 425.041214
if len(pi1)==0:
    pi1 = 10.004947
if len(po1)==0:
    po1 = 3
if len(f1)==0:
    f1 = 50
if len(pf1)==0:
    pf1 = 0.650188
if len(t1)==0:
    t1 = 55
if len(l1)==0:
    l1 = 85
if len(w1)==0:
    w1 = 100
if len(a1)==0:
    a1 = 5
    
pv,pi,po,f,pf,t,l,w,a= [float(pv1)],[float(pi1)],[float(po1)],[float(f1)],[float(pf1)],[float(t1)],[float(l1)],[float(w1)],[float(a1)]
df = pd.DataFrame(list(zip(pv,pi,po,f,pf,t,l,w,a)),columns = ['3 Phase Voltage in Volts','3 Phase Current in Amps','Poles','Frequency in Hz','Power Factor','Temperature in Celcius','% Load','Angular Velocity in rad/sec','Motor Age in Yrs'])

Transformation Pipeline and Model Sucessfully Loaded
[13:33:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
3 Phase Voltage in Volts: 
3 Phase Current in Amps: 
Poles: 
Frequency in Hz: 
Power Factor: 
Temperature in Celcius: 
% Load: 
Angular Velocity in rad/sec: 
Motor Age in Yrs: 


# Prediction

In [3]:
sd_v,sd_i,sd_t,sd_w,sd_age,a_v,a_i,a_t,a_w,a_age,a_pf,a_e = 14.450315,1.184078,14.724061,11.850687,3,425.041214,10.004947,55.033790,99.990990,5,0.65,80.63        

def warnings(pv1,pi1,t1,pf1,e):
    outstr = ""
    if pv1 > a_v + 1.2*sd_v : outstr += "High voltage ,"
    if t1 > a_t + 1.2*sd_t : outstr += "Overheating ,"
    if pv1*pi1*pf1*e/100 > 1.5*a_v*a_i*a_pf*a_e/100 : outstr += "Overloading ,"
    if len(outstr)==0 : outstr += "No Warnings"
    return outstr
    
    
def suggestions(pi1,t1,pf1,e):
    if  e >= 85.56 and t1 < 60:
        return "The asset is running efficiently any changes may make the asset less reliable. \n"
    elif e >= 85.56 and t1 >= 60:
        return "Asset cooling to" + str(t1*0.8) + "'C will give better efficiency. \n"
    else :
        if t1 < 60 and (pi1 != 8 or pf1 != 0.5):
            return "Efficiency can be increased by decreasing current to " + str(round((max(pi1*0.95,8)),4)) + " Amps or \n\tcorrecting the power factor to " + str(round(max(pf1*0.9,0.5),3)) + ". \n"
        elif t1 >= 60 and (pi1 != 8 or pf1 != 0.5 or t1 != 60):
            return "Efficiency can be increased by decreasing current to " + str(round((max(pi1*0.95,8)),4)) + " Amps, \n\tcorrecting the power factor to " + str(round(max(pf1*0.9,0.5),3)) + " or \n\treducing the temperature to " + str(round(t1*0.8,2)) + " 'C. \n"
        else : return "The asset is running efficiently any changes may make the asset less reliable. \n"

def relia(pv1, pi1, t1, a1, w1):     
    reliabilty = 5
    if pv1 > a_v + 1.2*sd_v:
        reliabilty -=1
    if pi1 > a_i + 1.2*sd_i:
        reliabilty -=1
    if t1 > a_t + 1.2*sd_t:
        reliabilty -=1
    if w1 > a_w + 1.2*sd_w:
        reliabilty -=1
    if a1 > a_age + sd_age:
        reliabilty -=1

    if reliabilty == 5:
        return 'Highly Reliable'
    if reliabilty == 3 or reliabilty == 4:
        return 'Moderately Reliable'
    if reliabilty == 1 or reliabilty == 2:
        return 'Less Reliable'
    if reliabilty == 0:
        return 'Not Reliable'

pred_new = predict_model(sih_model,data=df)
r = relia(pred_new['3 Phase Voltage in Volts'][0],pred_new['3 Phase Current in Amps'][0],pred_new['Temperature in Celcius'][0],pred_new['Motor Age in Yrs'][0],pred_new['Angular Velocity in rad/sec'][0])
e = pred_new['Label'][0]
c = 0
ctr = round(50 - (3500/e),0)
if ctr < 0 : ctr = 0
c = abs(ctr)

print("Effeciency: "+str(e)+"%")
print("Reliability: "+str(r))
print("Life Cyle: "+str(int(c))+" Years")
print("Warnings: "+warnings(pred_new['3 Phase Voltage in Volts'][0],pred_new['3 Phase Current in Amps'][0],pred_new['Temperature in Celcius'][0],pred_new['Power Factor'][0],e))
print("Suggestion: "+suggestions(pred_new['3 Phase Current in Amps'][0],pred_new['Temperature in Celcius'][0],pred_new['Power Factor'][0],e))

Effeciency: 79.3939%
Reliability: Highly Reliable
Life Cyle: 6 Years
Warnings: No Warnings
Suggestion: Efficiency can be increased by decreasing current to 9.5047 Amps or 
	correcting the power factor to 0.585. 

